**Chapter 6: Building Multi-Language Applications**

Theory becomes practice when we apply Dockerfile patterns to specific technology stacks. Each language ecosystem has unique requirements—from Python’s virtual environments to Java’s JVM tuning, from Node.js’s sprawling `node_modules` to Go’s static binaries. This chapter provides production-ready Dockerfile templates for the most common application types, demonstrating how to optimize builds, minimize attack surfaces, and handle dependencies idiomatically for each stack.

---

### 6.1 Python Applications

Python’s interpreted nature and dependency management via `pip` require careful attention to virtual environments, base image selection, and layer caching. We’ll cover both Django (full-stack) and Flask/FastAPI (microservice) patterns.

#### Django Application

**Project Structure:**
```
django-app/
├── Dockerfile
├── docker-compose.yml
├── requirements.txt
├── manage.py
└── myproject/
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py
```

**Production Dockerfile:**
```dockerfile
# syntax=docker/dockerfile:1
FROM python:3.11-slim as builder

# Security: Run as non-root
RUN groupadd -r django && useradd -r -g django django

# Install build dependencies
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    libpq-dev \
    && rm -rf /var/lib/apt/lists/*

# Create virtual environment
RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Install dependencies (layer caching)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Production stage
FROM python:3.11-slim

# Install runtime dependencies only
RUN apt-get update && apt-get install -y --no-install-recommends \
    libpq5 \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Copy virtual environment from builder
COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

# Create app user
RUN groupadd -r django && useradd -r -g django django

# Set working directory
WORKDIR /app

# Copy application code
COPY --chown=django:django . .

# Collect static files (Django specific)
RUN python manage.py collectstatic --noinput

# Switch to non-root user
USER django

# Health check
HEALTHCHECK --interval=30s --timeout=5s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:8000/health/ || exit 1

EXPOSE 8000

# Use gunicorn for production WSGI server
CMD ["gunicorn", "--bind", "0.0.0.0:8000", "--workers", "4", "--threads", "2", "--access-logfile", "-", "--error-logfile", "-", "myproject.wsgi:application"]
```

**Key Decisions Explained:**

1. **Multi-stage Build:** The builder stage installs `gcc` and build tools (hundreds of MB), while the final image contains only runtime libraries (`libpq5`), reducing size by ~60%.

2. **Virtual Environment:** Even in containers, virtual environments isolate Python packages from system Python, preventing conflicts with OS packages.

3. **Non-root User:** Django runs as UID 999, limiting damage if the application is compromised.

4. **Gunicorn:** The production WSGI server handles concurrency (workers/threads) and static file serving, unlike the development `runserver`.

**Requirements File (`requirements.txt`):**
```
Django>=4.2,<5.0
gunicorn>=21.0
psycopg2-binary>=2.9
dj-database-url>=2.0
whitenoise>=6.0  # For static files
```

**FastAPI/Flask Variant (Async):**
```dockerfile
# For FastAPI/Flask async applications
FROM python:3.11-alpine as builder

# Alpine requires different packages
RUN apk add --no-cache gcc musl-dev libffi-dev

RUN python -m venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

FROM python:3.11-alpine
RUN apk add --no-cache libffi curl

COPY --from=builder /opt/venv /opt/venv
ENV PATH="/opt/venv/bin:$PATH"

WORKDIR /app
COPY . .

# FastAPI specific
EXPOSE 8000
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000", "--workers", "4"]
```

**Key Takeaway:** Python containers must isolate dependencies in virtual environments and separate build tools from runtime. Use Alpine for minimal size, but Slim if you need glibc compatibility for compiled extensions like NumPy or Pandas.

---

### 6.2 Node.js Applications

Node.js applications face unique challenges: huge `node_modules` directories, native addon compilation, and the need to separate development dependencies from production.

#### Express API (Production-Optimized)

**Project Structure:**
```
node-api/
├── Dockerfile
├── package.json
├── package-lock.json
└── src/
    └── server.js
```

**Multi-stage Dockerfile:**
```dockerfile
# syntax=docker/dockerfile:1
# Stage 1: Dependencies
FROM node:20-alpine AS deps
RUN apk add --no-cache libc6-compat python3 make g++

WORKDIR /app

# Copy package files
COPY package.json package-lock.json* ./

# Install dependencies (including devDependencies for build)
RUN npm ci

# Stage 2: Builder
FROM node:20-alpine AS builder
WORKDIR /app

# Copy dependencies from deps stage
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Build application (if using TypeScript or bundler)
RUN npm run build

# Remove devDependencies
RUN npm prune --production

# Stage 3: Runner (Production)
FROM node:20-alpine AS runner
WORKDIR /app

# Security: Create non-root user
RUN addgroup --system --gid 1001 nodejs
RUN adduser --system --uid 1001 nodejs

# Copy only necessary files
COPY --from=builder --chown=nodejs:nodejs /app/dist ./dist
COPY --from=builder --chown=nodejs:nodejs /app/node_modules ./node_modules
COPY --from=builder --chown=nodejs:nodejs /app/package.json ./package.json

# Set environment
ENV NODE_ENV=production
ENV PORT=3000

USER nodejs

EXPOSE 3000

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=10s \
    CMD node -e "require('http').get('http://localhost:3000/health', (r) => {process.exit(r.statusCode === 200 ? 0 : 1)})"

CMD ["node", "dist/server.js"]
```

**Critical Optimizations:**

1. **Three-Stage Build:**
   - `deps`: Install all dependencies (including native compilation tools)
   - `builder`: Compile TypeScript/build assets, then prune devDependencies
   - `runner`: Only production code and dependencies (~1/10th the size)

2. **Package-lock.json:** Always copy `package-lock.json` (or `yarn.lock`, `pnpm-lock.yaml`) to ensure deterministic, reproducible installs.

3. **Native Modules:** The `deps` stage includes `python3` and `make` for compiling native addons (bcrypt, sqlite3), but these don’t exist in the final `runner` stage.

**Next.js/React Frontend Variant:**
```dockerfile
# For Next.js applications
FROM node:20-alpine AS base

# Install dependencies only when needed
FROM base AS deps
RUN apk add --no-cache libc6-compat
WORKDIR /app
COPY package.json package-lock.json* ./
RUN npm ci

# Rebuild source only when needed
FROM base AS builder
WORKDIR /app
COPY --from=deps /app/node_modules ./node_modules
COPY . .

# Build static files
ENV NEXT_TELEMETRY_DISABLED=1
RUN npm run build

# Production image
FROM base AS runner
WORKDIR /app

ENV NODE_ENV=production
ENV NEXT_TELEMETRY_DISABLED=1

RUN addgroup --system --gid 1001 nodejs
RUN adduser --system --uid 1001 nextjs

# Copy only standalone output (Next.js 12.1+)
COPY --from=builder --chown=nextjs:nodejs /app/.next/standalone ./
COPY --from=builder --chown=nextjs:nodejs /app/.next/static ./.next/static
COPY --from=builder --chown=nextjs:nodejs /app/public ./public

USER nextjs

EXPOSE 3000

ENV PORT=3000
ENV HOSTNAME="0.0.0.0"

CMD ["node", "server.js"]
```

**Key Takeaway:** Node.js containers must use multi-stage builds to exclude `node_modules` devDependencies and build tools. The final image should contain only the compiled JavaScript and production dependencies, typically reducing size from 1GB+ to under 200MB.

---

### 6.3 Java/Spring Boot Applications

Java applications require JVM tuning and careful handling of JAR files. Spring Boot’s layered JAR feature allows Docker to cache dependency layers separately from application code.

#### Spring Boot with Layered JARs

**Dockerfile:**
```dockerfile
# syntax=docker/dockerfile:1
FROM eclipse-temurin:17-jdk-alpine as builder
WORKDIR /application

# Copy Maven wrapper and pom
COPY mvnw .
COPY .mvn .mvn
COPY pom.xml .

# Download dependencies (cached layer)
RUN ./mvnw dependency:go-offline -B

# Copy source and build
COPY src src
RUN ./mvnw clean package -DskipTests -B

# Extract layers from Spring Boot fat JAR
RUN java -Djarmode=layertools -jar target/*.jar extract

# Production stage
FROM eclipse-temurin:17-jre-alpine
WORKDIR /application

# Create non-root user
RUN addgroup -S spring && adduser -S spring -G spring
USER spring:spring

# Copy extracted layers (dependencies first for caching)
COPY --from=builder --chown=spring:spring application/dependencies/ ./
COPY --from=builder --chown=spring:spring application/spring-boot-loader/ ./
COPY --from=builder --chown=spring:spring application/snapshot-dependencies/ ./
COPY --from=builder --chown=spring:spring application/application/ ./

# JVM options for containers
ENV JAVA_OPTS="-XX:+UseContainerSupport -XX:MaxRAMPercentage=75.0 -XX:InitialRAMPercentage=50.0"

EXPOSE 8080

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=60s \
    CMD wget --quiet --tries=1 --spider http://localhost:8080/actuator/health || exit 1

ENTRYPOINT ["sh", "-c", "java $JAVA_OPTS org.springframework.boot.loader.launch.JarLauncher"]
```

**Layer Optimization Explained:**

Spring Boot 2.3+ creates JARs with layers:
1. `dependencies`: Third-party libraries (changes rarely)
2. `spring-boot-loader`: Boot loader code (changes rarely)
3. `snapshot-dependencies`: Snapshot dependencies (changes occasionally)
4. `application`: Your code (changes frequently)

By copying these as separate layers, Docker caches `dependencies` and `snapshot-dependencies`, only rebuilding the thin `application` layer when your code changes.

**Maven Configuration (`pom.xml` snippet):**
```xml
<build>
    <plugins>
        <plugin>
            <groupId>org.springframework.boot</groupId>
            <artifactId>spring-boot-maven-plugin</artifactId>
            <configuration>
                <layers>
                    <enabled>true</enabled>
                </layers>
            </configuration>
        </plugin>
    </plugins>
</build>
```

**Alternative: Distroless Java**
For maximum security, use Google's Distroless images:

```dockerfile
FROM eclipse-temurin:17-jdk-alpine AS builder
WORKDIR /app
COPY . .
RUN ./mvnw clean package -DskipTests

FROM gcr.io/distroless/java17-debian12:nonroot
COPY --from=builder /app/target/*.jar /app.jar
EXPOSE 8080
USER nonroot:nonroot
ENTRYPOINT ["java", "-jar", "/app.jar"]
```

**Key Takeaway:** Java containers should use JRE (not JDK) bases for production, implement layered JARs to optimize build caching, and tune JVM heap settings for containerized environments (`-XX:MaxRAMPercentage`).

---

### 6.4 Go Applications

Go produces static binaries, making it ideal for minimal containers—often using `scratch` or Distroless bases with no OS overhead.

#### Standard Go Service

**Project Structure:**
```
go-api/
├── Dockerfile
├── go.mod
├── go.sum
└── cmd/
    └── server/
        └── main.go
```

**Optimized Dockerfile:**
```dockerfile
# syntax=docker/dockerfile:1
# Build stage
FROM golang:1.21-alpine AS builder

# Install git and ca-certificates for HTTPS
RUN apk add --no-cache git ca-certificates tzdata

# Create non-root user for final image
RUN adduser -D -g '' appuser

WORKDIR /build

# Copy go mod files first (dependency caching)
COPY go.mod go.sum ./
RUN go mod download && go mod verify

# Copy source
COPY . .

# Build static binary
# CGO_ENABLED=0 for static linking
# -ldflags="-w -s" strips debug info
RUN CGO_ENABLED=0 GOOS=linux GOARCH=amd64 go build \
    -ldflags="-w -s -extldflags '-static'" \
    -a -installsuffix cgo \
    -o /build/server \
    ./cmd/server

# Production stage - Distroless
FROM gcr.io/distroless/static:nonroot

# Copy CA certs for HTTPS
COPY --from=builder /etc/ssl/certs/ca-certificates.crt /etc/ssl/certs/

# Copy user from builder (for Distroless compatibility)
COPY --from=builder /etc/passwd /etc/passwd

# Copy binary
COPY --from=builder /build/server /server

# Use non-root user
USER nonroot:nonroot

EXPOSE 8080

# Health check (requires Go app to implement /health)
HEALTHCHECK --interval=30s --timeout=3s \
    CMD ["/server", "-health-check"] || exit 1

ENTRYPOINT ["/server"]
```

**Key Optimizations:**

1. **Static Binary:** `CGO_ENABLED=0` produces a binary with no C library dependencies, runnable in `scratch` or Distroless images.

2. **Size Reduction:** `-ldflags="-w -s"` strips debug symbols and DWARF tables, reducing binary size by ~30%.

3. **Dependency Caching:** Copying `go.mod`/`go.sum` before source code allows `go mod download` to cache, only re-running when dependencies change.

**Alternative: Scratch Base**
If you don't need HTTPS certificates or timezone data:

```dockerfile
FROM scratch
COPY --from=builder /build/server /server
EXPOSE 8080
USER 65532:65532  # Must use numeric UID for scratch
ENTRYPOINT ["/server"]
```

**Key Takeaway:** Go’s static compilation enables the smallest, most secure containers possible. A typical Go application container is under 20MB, with no shell, package manager, or unnecessary utilities—minimizing attack surface to the absolute minimum.

---

### 6.5 Static Sites with Nginx

For React, Vue, Angular, or plain HTML sites, Nginx provides an efficient, production-grade web server.

#### React/Vue Production Build

**Dockerfile:**
```dockerfile
# syntax=docker/dockerfile:1
# Build stage
FROM node:20-alpine AS builder
WORKDIR /app

COPY package*.json ./
RUN npm ci

COPY . .
RUN npm run build

# Production stage
FROM nginx:1.25-alpine

# Remove default nginx config
RUN rm /etc/nginx/conf.d/default.conf

# Copy custom nginx config
COPY nginx.conf /etc/nginx/conf.d/

# Copy built assets from builder
COPY --from=builder /app/dist /usr/share/nginx/html

# Create non-root user (nginx runs as user 'nginx' by default)
RUN chown -R nginx:nginx /usr/share/nginx/html && \
    chown -R nginx:nginx /var/cache/nginx && \
    chown -R nginx:nginx /var/log/nginx && \
    chown -R nginx:nginx /etc/nginx/conf.d

# Switch to non-root
USER nginx

EXPOSE 80

# Health check
HEALTHCHECK --interval=30s --timeout=3s \
    CMD wget --quiet --tries=1 --spider http://localhost/ || exit 1

CMD ["nginx", "-g", "daemon off;"]
```

**Nginx Configuration (`nginx.conf`):**
```nginx
server {
    listen 80;
    server_name localhost;
    root /usr/share/nginx/html;
    index index.html;

    # Gzip compression
    gzip on;
    gzip_vary on;
    gzip_min_length 1024;
    gzip_types text/plain text/css application/json application/javascript text/xml;

    # Cache static assets
    location ~* \.(js|css|png|jpg|jpeg|gif|ico|svg)$ {
        expires 1y;
        add_header Cache-Control "public, immutable";
    }

    # Handle React Router / Vue Router
    location / {
        try_files $uri $uri/ /index.html;
    }

    # Security headers
    add_header X-Frame-Options "SAMEORIGIN";
    add_header X-Content-Type-Options "nosniff";
    add_header X-XSS-Protection "1; mode=block";
}
```

**Key Takeaway:** Static sites should use multi-stage builds to compile assets with Node.js, then serve with Nginx. Configure proper caching headers and client-side routing fallbacks for SPA frameworks.

---

### 6.6 Database Containers

While managed databases are preferred for production, containerized databases are essential for local development and CI/CD testing.

#### PostgreSQL with Initialization

**Dockerfile (Custom Postgres):**
```dockerfile
FROM postgres:15-alpine

# Install additional extensions if needed
RUN apk add --no-cache postgresql15-contrib

# Copy initialization scripts
COPY ./init-scripts/ /docker-entrypoint-initdb.d/

# Copy custom configuration
COPY postgresql.conf /etc/postgresql/postgresql.conf

# Non-root user handling (Postgres image already uses postgres user)
USER postgres

EXPOSE 5432

# Default CMD from base image handles initialization
```

**Initialization Script (`init-scripts/01-init.sql`):**
```sql
-- Runs automatically on first container start
CREATE DATABASE myapp;
CREATE USER appuser WITH ENCRYPTED PASSWORD 'changeme';
GRANT ALL PRIVILEGES ON DATABASE myapp TO appuser;

-- Initialize schema
\c myapp;
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    email VARCHAR(255) UNIQUE NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

**Docker Compose for Development:**
```yaml
version: '3.8'
services:
  postgres:
    build:
      context: ./postgres
    environment:
      POSTGRES_USER: root
      POSTGRES_PASSWORD: rootpassword
      POSTGRES_DB: myapp
    volumes:
      - postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -U root -d myapp"]
      interval: 5s
      timeout: 5s
      retries: 5

  redis:
    image: redis:7-alpine
    command: redis-server --appendonly yes --requirepass redispass
    volumes:
      - redis_data:/data
    ports:
      - "6379:6379"
    healthcheck:
      test: ["CMD", "redis-cli", "ping"]
      interval: 5s

volumes:
  postgres_data:
  redis_data:
```

**Key Takeaway:** Database containers should mount volumes for persistence, use initialization scripts for schema setup, and implement health checks. Never store production data in containers without volumes, and never commit database passwords to images—use environment variables.

---

### 6.7 Handling Dependencies

Dependency management varies by language but follows common principles: cache aggressively, pin versions, and audit for security.

#### Language-Specific Strategies

**Python (pip):**
```dockerfile
# Use requirements.txt with hashes for security
RUN pip install --no-cache-dir --require-hashes -r requirements.txt

# Or use Poetry for lock files
COPY poetry.lock pyproject.toml ./
RUN poetry install --no-dev --no-interaction --no-ansi
```

**Node.js (npm/yarn/pnpm):**
```dockerfile
# Use npm ci (clean install) instead of npm install
RUN npm ci --only=production

# For Yarn
COPY yarn.lock ./
RUN yarn install --frozen-lockfile --production

# For pnpm (fastest, smallest node_modules)
COPY pnpm-lock.yaml ./
RUN pnpm install --frozen-lockfile --prod
```

**Java (Maven/Gradle):**
```dockerfile
# Maven offline mode after initial download
RUN ./mvnw dependency:go-offline -B
RUN ./mvnw package -o -DskipTests  # Offline mode

# Gradle
RUN ./gradlew dependencies --no-daemon
RUN ./gradlew build --no-daemon -x test
```

**Go (Modules):**
```dockerfile
# Vendor mode for reproducible builds
RUN go mod download
RUN go mod verify
# Or use vendoring: COPY vendor ./
```

#### Security Scanning in CI

Always scan dependencies before production:

```dockerfile
# Add to your CI pipeline (not production image)
FROM aquasec/trivy:latest as scanner
COPY --from=builder /app /scan
RUN trivy filesystem --exit-code 1 --severity HIGH,CRITICAL /scan
```

#### Private Repository Authentication

For private npm, PyPI, or Maven repositories:

```dockerfile
# NPM private registry
ARG NPM_TOKEN
RUN npm config set //registry.npmjs.org/:_authToken=${NPM_TOKEN}
RUN npm ci
RUN rm -f .npmrc  # Remove token from image

# Or use BuildKit secrets (recommended)
RUN --mount=type=secret,id=npmrc,target=/root/.npmrc npm ci
```

**Key Takeaway:** Dependencies are the largest attack surface in containerized applications. Pin versions using lock files, scan for vulnerabilities, and use multi-stage builds to ensure build tools don't reach production. Treat dependency updates as critical infrastructure changes requiring CI/CD validation.

---

### Chapter Summary and Preview

In this chapter, you applied Dockerfile fundamentals to real-world technology stacks. You built **Python** containers with virtual environments and multi-stage separation of build tools, optimized **Node.js** images using three-stage builds to eliminate devDependencies, implemented **Java/Spring Boot** layered JARs for efficient dependency caching, created minimal **Go** containers using Distroless bases with static binaries, served **static sites** with Nginx and proper SPA routing configuration, and containerized **databases** with initialization scripts and persistent volumes. We established **dependency management** patterns emphasizing version pinning, lock files, and security scanning.

You now possess production-ready Dockerfile templates for the majority of modern application types. These patterns directly translate to the CI/CD pipelines we will construct—where these Dockerfiles will be executed automatically, producing the immutable artifacts that flow through staging to production.

In **Chapter 7: Docker Compose for Local Development**, we move from single containers to multi-container applications. You will learn to orchestrate your web application alongside its database, cache, and message queue using Docker Compose. We will cover networking between services, volume persistence, environment-specific configurations, and the critical distinction between development convenience and production parity. This chapter bridges the gap between `docker run` commands and full Kubernetes orchestration, teaching you to replicate production architectures on your local machine for efficient development and testing.